# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

I'm going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries.

In [1]:
import pandas as pd
import numpy as np
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('/Users/sagarprasad/Downloads/algorithmic-trading-python-master/starter_files/sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe.

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2728017141680,
 'week52high': 197.7,
 'week52low': 123.48,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.026026099601385244,
 'sharesOutstanding': 15634232000,
 'float': 0,
 'avg10Volume': 52299622,
 'avg30Volume': 56135299,
 'day200MovingAvg': 170.94,
 'day50MovingAvg': 187.35,
 'employees': 164000,
 'ttmEPS': 5.95,
 'ttmDividendRate': 0.936842139930527,
 'dividendYield': 0.005369030545764954,
 'nextDividendDate': '',
 'exDividendDate': '2023-08-11',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 28.78869925791473,
 'beta': 1.290902798699239,
 'maxChangePercent': 67.0219865897396,
 'year5ChangePercent': 2.391197540317722,
 'year2ChangePercent': 0.1942110637505714,
 'year1ChangePercent': 0.026026099601385244,
 'ytdChangePercent': 0.3505041655186081,
 'month6ChangePercent': 0.1484992631411972,
 'month3ChangePercent': -0.0011328775490461185,
 'month1ChangePercent': -0.09407376064005146,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.026026099601385244

## Executing A Batch API Call & Building Our DataFrame

it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
import warnings
warnings.filterwarnings("ignore")

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data)
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else :
            final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
            index = my_columns),
            ignore_index = True
            )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,118.86,-0.12761,N/A
1,AAL,15.16,0.061397,N/A
2,AAP,68.20,-0.650552,N/A
3,AAPL,175.84,0.026026,N/A
4,ABBV,150.16,0.101382,N/A
...,...,...,...,...
496,YUM,128.80,0.12264,N/A
497,ZBH,120.35,0.060681,N/A
498,ZBRA,267.53,-0.171828,N/A
499,ZION,34.53,-0.369595,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,388,RCL,99.27,1.501892,N/A
2,343,NVDA,469.67,1.428107,N/A
3,193,FTI,18.32,1.15703,N/A
4,368,PHM,77.96,0.897579,N/A
5,197,GE,112.02,0.834728,N/A
6,281,LLY,550.05,0.711245,N/A
7,330,NFLX,408.29,0.677434,N/A
8,132,DHI,115.94,0.594798,N/A
9,410,SLB,57.44,0.584645,N/A


## Calculating the Number of Shares to Buy

we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \nPlease try again:")
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)
portfolio_input()

Enter the value of your portfolio:10000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,2502
1,388,RCL,99.27,1.501892,2014
2,343,NVDA,469.67,1.428107,425
3,193,FTI,18.32,1.15703,10917
4,368,PHM,77.96,0.897579,2565
5,197,GE,112.02,0.834728,1785
6,281,LLY,550.05,0.711245,363
7,330,NFLX,408.29,0.677434,489
8,132,DHI,115.94,0.594798,1725
9,410,SLB,57.44,0.584645,3481


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else :
            hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
            index = hqm_columns),
            ignore_index = True
            )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.86,N/A,-0.12761,N/A,-0.192821,N/A,-0.072882,N/A,-0.042553,N/A,N/A
1,AAL,15.16,N/A,0.061397,N/A,-0.080685,N/A,0.014845,N/A,-0.137615,N/A,N/A
2,AAP,68.20,N/A,-0.650552,N/A,-0.526937,N/A,-0.403137,N/A,-0.002571,N/A,N/A
3,AAPL,175.84,N/A,0.026026,N/A,0.148499,N/A,-0.001133,N/A,-0.094074,N/A,N/A
4,ABBV,150.16,N/A,0.101382,N/A,0.012497,N/A,0.046134,N/A,0.055911,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.80,N/A,0.12264,N/A,-0.017551,N/A,-0.061886,N/A,-0.04975,N/A,N/A
497,ZBH,120.35,N/A,0.060681,N/A,-0.05052,N/A,-0.115717,N/A,-0.164997,N/A,N/A
498,ZBRA,267.53,N/A,-0.171828,N/A,-0.15686,N/A,-0.03141,N/A,-0.106915,N/A,N/A
499,ZION,34.53,N/A,-0.369595,N/A,-0.30488,N/A,0.312408,N/A,-0.073004,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
        'One-Year',
        'Six-Month',
        'Three-Month',
        'One-Month'
        ]
hqm_dataframe.fillna(0, inplace= True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe       

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.86,N/A,-0.127610,0.251497,-0.192821,0.105788,-0.072882,0.135729,-0.042553,0.493014,N/A
1,AAL,15.16,N/A,0.061397,0.616766,-0.080685,0.287425,0.014845,0.46507,-0.137615,0.077844,N/A
2,AAP,68.20,N/A,-0.650552,0.003992,-0.526937,0.005988,-0.403137,0.003992,-0.002571,0.676647,N/A
3,AAPL,175.84,N/A,0.026026,0.546906,0.148499,0.864271,-0.001133,0.367265,-0.094074,0.177645,N/A
4,ABBV,150.16,N/A,0.101382,0.700599,0.012497,0.580838,0.046134,0.580838,0.055911,0.894212,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,128.80,N/A,0.122640,0.736527,-0.017551,0.487026,-0.061886,0.161677,-0.049750,0.443114,N/A
497,ZBH,120.35,N/A,0.060681,0.61477,-0.050520,0.393214,-0.115717,0.083832,-0.164997,0.03992,N/A
498,ZBRA,267.53,N/A,-0.171828,0.169661,-0.156860,0.143713,-0.031410,0.255489,-0.106915,0.133733,N/A
499,ZION,34.53,N/A,-0.369595,0.03992,-0.304880,0.023952,0.312408,0.966068,-0.073004,0.293413,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LLY,550.05,N/A,0.711245,0.988024,0.671820,0.996008,0.237853,0.938124,0.180633,0.996008,0.979541
1,LB,79.92,N/A,2.281287,1.0,0.815831,0.998004,0.215421,0.918164,0.079271,0.94012,0.964072
2,MPC,142.85,N/A,0.474625,0.966068,0.194662,0.916168,0.317686,0.972056,0.169851,0.994012,0.962076
3,FTI,18.32,N/A,1.157030,0.994012,0.304065,0.96008,0.316284,0.97006,0.045792,0.866267,0.947605
4,COG,22.25,N/A,0.340483,0.92016,0.180873,0.896208,0.255629,0.952096,0.249298,0.998004,0.941617
5,ANET,184.67,N/A,0.388769,0.938124,0.306084,0.964072,0.254534,0.9501,0.053264,0.886228,0.934631
6,APA,43.90,N/A,0.261990,0.866267,0.183977,0.902196,0.341126,0.974052,0.164604,0.992016,0.933633
7,PSX,113.33,N/A,0.343237,0.922156,0.171190,0.886228,0.214545,0.916168,0.108206,0.972056,0.924152
8,PHM,77.96,N/A,0.897579,0.992016,0.467768,0.994012,0.148771,0.852295,0.022962,0.816367,0.913673
9,HES,153.67,N/A,0.350265,0.924152,0.155340,0.874251,0.168551,0.88024,0.108451,0.974052,0.913174


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portfolio_input()

Enter the value of your portfolio:10000000


In [16]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy']= math.floor(position_size / hqm_dataframe.loc[i,'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LLY,550.05,363,0.711245,0.988024,0.671820,0.996008,0.237853,0.938124,0.180633,0.996008,0.979541
1,LB,79.92,2502,2.281287,1.0,0.815831,0.998004,0.215421,0.918164,0.079271,0.94012,0.964072
2,MPC,142.85,1400,0.474625,0.966068,0.194662,0.916168,0.317686,0.972056,0.169851,0.994012,0.962076
3,FTI,18.32,10917,1.157030,0.994012,0.304065,0.96008,0.316284,0.97006,0.045792,0.866267,0.947605
4,COG,22.25,8988,0.340483,0.92016,0.180873,0.896208,0.255629,0.952096,0.249298,0.998004,0.941617
5,ANET,184.67,1083,0.388769,0.938124,0.306084,0.964072,0.254534,0.9501,0.053264,0.886228,0.934631
6,APA,43.90,4555,0.261990,0.866267,0.183977,0.902196,0.341126,0.974052,0.164604,0.992016,0.933633
7,PSX,113.33,1764,0.343237,0.922156,0.171190,0.886228,0.214545,0.916168,0.108206,0.972056,0.924152
8,PHM,77.96,2565,0.897579,0.992016,0.467768,0.994012,0.148771,0.852295,0.022962,0.816367,0.913673
9,HES,153.67,1301,0.350265,0.924152,0.155340,0.874251,0.168551,0.88024,0.108451,0.974052,0.913174


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase


In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [20]:
writer.save()